In [1]:
from twython import Twython
import time
import math
import json

In [2]:
def twitter_oauth_login():
    API_KEY = 'T4afJ8JtkGuDdkCkN0q1zpxs9'
    API_SECRET = 'Rl5tXrGt0rqanaBSYOIc3LIjNFbVlImKPYATXl7i7nGaFxHdk6'
    ACCESS_TOKEN = '215666419-NsFtzS9ajR3YYfUx6scDAzbCI2oDp6attMoMf3IG'
    ACCESS_TOKEN_SECRET = 'NiTMzyhQrGRQCFy6UxNbnant09Lwn2AD8UADF1KRbbOpX'
    
    twitter = Twython(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    return twitter

In [3]:
twitter = twitter_oauth_login()

In [4]:
friends_ids = twitter.get_friends_ids(count=5000)
friends_ids = friends_ids['ids']

followers_ids = twitter.get_followers_ids(count=5000)
followers_ids = followers_ids['ids']

friends_set = set(friends_ids)
followers_set = set(followers_ids)

In [12]:
print(len(friends_set.union(followers_set)))

292


In [13]:
print(len(friends_set & followers_set))

27


In [5]:
def pull_user_profiles(ids):
    users = []
    for i in range(0, len(ids), 100):
        batch = ids[i:i+100]
        users += twitter.lookup_user(user_id=batch)
        print(twitter.get_lastfunction_header('x-rate-limit-remaining'))
        
    return users

In [6]:
friends_profiles = pull_user_profiles(friends_ids)
followers_profiles = pull_user_profiles(followers_ids)

179
178
177
176


In [18]:
friends_screen_names = [p['screen_name'] for p in friends_profiles if 'screen_name' in p]
print(friends_screen_names)

[u'Medium', u'Quai_36', u'neilhalloran', u'MITxULab', u'ottoscharmer1', u'thedatainc', u'BIPBit', u'dataphilosopher', u'DataconomyMedia', u'axellelemaire', u'leszebres', u'MutinerieCrew', u'DataIsBeautiful', u'kaggle', u'antgoldbloom', u'brainmagazine', u'ronez', u'USBEKetRICA', u'3dprintindustry', u'OKFestival', u'pierre_alonso', u'csoghoian', u'skillshare', u'BerlinTechMeet', u'luciemontel', u'FrenchTechClub', u'nicolaspatte', u'TechCrunch', u'techreview', u'medialab', u'o0omeli', u'CNNum', u'frprosper', u'btabaka', u'dataveyes', u'we_do_data', u'pauljoannon', u'Toutenrab', u'vied12', u'annelisebouyer', u'jplusplus_', u'pirhoo', u'AxelleBalvay', u'LaFrenchTech', u'SchoolOfData', u'OKFN', u'bigdata', u'sup_numerique', u'peteskomoroch', u'DataJunkie', u'RNumerique', u'SocialNextwork', u'startupacademy', u'MehdiBenkler', u'salome_k', u'arthackday', u'Paris_Incub', u'baborlelefan', u'creativeapps', u'ungeekaparis', u'reservoirfunds', u'theleanstartup', u'goleanfr', u'co_up', u'rhizome', 

In [22]:
rate_limit_window = 15*60

def pull_user_profiles_limit_aware(ids):
    
    users = []
    start_time = time.time()

    for i in range(0, len(ids), 100):
        batch = ids[i:i+100]
        users += twitter.lookup_user(user_id=batch)
        
        calls_left = float(twitter.get_lastfunction_header('x-rate-limit-remaining'))
        time_remaining_in_window = rate_limit_window - (time.time() - start_time)
        sleep_duration = math.ceil(time_remaining_in_window/calls_left)
        print ('Sleeping for: ' + str(sleep_duration) + ' seconds; ' + str(calls_left) + ' API calls remaining.')
        time.sleep(sleep_duration)
        
    return users

In [7]:
def save_json(filename, data):
    with open(filename, 'wb') as outfile:
        json.dump(data, outfile)
        
def load_json(filename):
    with open(filename) as infile:
        data = json.load(infile)
    return data

In [8]:
fname = 'test_followers_profiles.json'
save_json(fname, followers_profiles)